importing library

In [78]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



/Users/soroushrasti/miniconda2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


Make a GET request to fetch the raw HTML content


In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(url).text


# Step 2: Parse the html content
soup = BeautifulSoup(html_content)
#print(soup.prettify()) # print the parsed data of html



#Get the table having the class wikitable
table = soup.find("table", attrs={"class": "wikitable"})
table_data = table.tbody.find_all("tr")  # contains 2 rows
print ( len(table_data))






putiting to data frame

In [ ]:
# Get all the headings 
headings = []
for th in table_data[0].find_all("th"):
    headings.append(th.text)

#tranforming to dictionary 
data={}
for i in range(1,len(table_data)): 
    content=[]
    for td in table_data[i].find_all("td"):
        content.append(td.text)
    data.setdefault(headings[0],[]).append(content[0])
    data.setdefault(headings[1],[]).append(content[1])
    data.setdefault(headings[2],[]).append(content[2])

# 
data=pd.DataFrame.from_dict(data)
# remove /n
data = data.replace('\n',' ', regex=True)

# delete rows with 'Not assigned' in the 'Borough' coloumn
indexNames = data[ data['Borough'] == 'Not assigned' ].index
data.drop(indexNames , inplace=True)

# repalce if Neighbourhood has 'Not assigned'
indexNames = data[ data['Neighbourhood\n'].str.strip() == 'Not assigned' ].index
data['Neighbourhood\n'][indexNames]=data['Borough'][indexNames]
#




combine two Neighbourhood with the same Postcode and Borough 
and make into separate columns

In [ ]:

data1=data.groupby(['Postcode','Borough'],as_index = True)['Neighbourhood\n'].apply(lambda x: pd.Series(list(x))).unstack().reset_index()

data1.columns=['Postcode','Borough','Neighbourhood1','Neighbourhood2','Neighbourhood3','Neighbourhood4','Neighbourhood5','Neighbourhood6','Neighbourhood7','Neighbourhood8']
data1['Neighbourhoodall'] = data1[['Neighbourhood1', 'Neighbourhood2','Neighbourhood3','Neighbourhood4','Neighbourhood5','Neighbourhood6','Neighbourhood7','Neighbourhood8']].apply(lambda x: ','.join(x.fillna('').map(str)), axis=1)
data1['Neighbourhoodall'] = data1['Neighbourhoodall'].str.strip(',')
data1['Neighbourhoodall']
datawithcomma = data1[['Postcode','Borough','Neighbourhoodall']].copy()
datawithcomma



#I here I extract the Borough that constains Toronto


In [73]:
toronto_data = datawithcomma[datawithcomma['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhoodall
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"The Danforth West ,Riverdale"
2,M4L,East Toronto,"The Beaches West ,India Bazaar"
3,M4M,East Toronto,Studio District
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North
6,M4R,Central Toronto,North Toronto West
7,M4S,Central Toronto,Davisville
8,M4T,Central Toronto,"Moore Park ,Summerhill East"
9,M4V,Central Toronto,"Deer Park ,Forest Hill SE ,Rathnelly ,South Hi..."


# Because I could not get access to CSV file provided in the previous exmaple, I just used NewYork Latitude and Longitude for the Toronto data and did K-means and map analyze on that

In [54]:
#!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
#with open('newyork_data.json') as json_data:
#    newyork_data = json.load(json_data)

newyork_data={'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.3',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82780644716412, 40.887555677350775]},
   'geometry_name': 'geom',
   'properties': {'name': 'Eastchester',
    'stacked': 1,
    'annoline1': 'Eastchester',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82780644716412,
     40.887555677350775,
     -73.82780644716412,
     40.887555677350775]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.4',
   'geometry': {'type': 'Point',
    'coordinates': [-73.90564259591682, 40.89543742690383]},
   'geometry_name': 'geom',
   'properties': {'name': 'Fieldston',
    'stacked': 1,
    'annoline1': 'Fieldston',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.90564259591682,
     40.89543742690383,
     -73.90564259591682,
     40.89543742690383]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.5',
   'geometry': {'type': 'Point',
    'coordinates': [-73.9125854610857, 40.890834493891305]},
   'geometry_name': 'geom',
   'properties': {'name': 'Riverdale',
    'stacked': 1,
    'annoline1': 'Riverdale',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.9125854610857,
     40.890834493891305,
     -73.9125854610857,
     40.890834493891305]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.6',
   'geometry': {'type': 'Point',
    'coordinates': [-73.90281798724604, 40.88168737120521]},
   'geometry_name': 'geom',
   'properties': {'name': 'Kingsbridge',
    'stacked': 1,
    'annoline1': 'Kingsbridge',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.90281798724604,
     40.88168737120521,
     -73.90281798724604,
     40.88168737120521]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.7',
   'geometry': {'type': 'Point',
    'coordinates': [-73.91065965862981, 40.87655077879964]},
   'geometry_name': 'geom',
   'properties': {'name': 'Marble Hill',
    'stacked': 2,
    'annoline1': 'Marble',
    'annoline2': 'Hill',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Manhattan',
    'bbox': [-73.91065965862981,
     40.87655077879964,
     -73.91065965862981,
     40.87655077879964]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.8',
   'geometry': {'type': 'Point',
    'coordinates': [-73.86731496814176, 40.89827261213805]},
   'geometry_name': 'geom',
   'properties': {'name': 'Woodlawn',
    'stacked': 1,
    'annoline1': 'Woodlawn',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.86731496814176,
     40.89827261213805,
     -73.86731496814176,
     40.89827261213805]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.9',
   'geometry': {'type': 'Point',
    'coordinates': [-73.8793907395681, 40.87722415599446]},
   'geometry_name': 'geom',
   'properties': {'name': 'Norwood',
    'stacked': 1,
    'annoline1': 'Norwood',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.8793907395681,
     40.87722415599446,
     -73.8793907395681,
     40.87722415599446]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.10',
   'geometry': {'type': 'Point',
    'coordinates': [-73.85744642974207, 40.88103887819211]},
   'geometry_name': 'geom',
   'properties': {'name': 'Williamsbridge',
    'stacked': 1,
    'annoline1': 'Williamsbridge',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.85744642974207,
     40.88103887819211,
     -73.85744642974207,
     40.88103887819211]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.11',
   'geometry': {'type': 'Point',
    'coordinates': [-73.83579759808117, 40.866858107252696]},
   'geometry_name': 'geom',
   'properties': {'name': 'Baychester',
    'stacked': 1,
    'annoline1': 'Baychester',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.83579759808117,
     40.866858107252696,
     -73.83579759808117,
     40.866858107252696]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.12',
   'geometry': {'type': 'Point',
    'coordinates': [-73.85475564017999, 40.85741349808865]},
   'geometry_name': 'geom',
   'properties': {'name': 'Pelham Parkway',
    'stacked': 1,
    'annoline1': 'Pelham Parkway',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.85475564017999,
     40.85741349808865,
     -73.85475564017999,
     40.85741349808865]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.13',
   'geometry': {'type': 'Point',
    'coordinates': [-73.78648845267413, 40.84724670491813]},
   'geometry_name': 'geom',
   'properties': {'name': 'City Island',
    'stacked': 2,
    'annoline1': 'City',
    'annoline2': 'Island',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.78648845267413,
     40.84724670491813,
     -73.78648845267413,
     40.84724670491813]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.14',
   'geometry': {'type': 'Point',
    'coordinates': [-73.8855121841913, 40.870185164975325]},
   'geometry_name': 'geom',
   'properties': {'name': 'Bedford Park',
    'stacked': 2,
    'annoline1': 'Bedford',
    'annoline2': 'Park',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.8855121841913,
     40.870185164975325,
     -73.8855121841913,
     40.870185164975325]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.15',
   'geometry': {'type': 'Point',
    'coordinates': [-73.9104159619131, 40.85572707719664]},
   'geometry_name': 'geom',
   'properties': {'name': 'University Heights',
    'stacked': 2,
    'annoline1': 'University',
    'annoline2': 'Heights',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.9104159619131,
     40.85572707719664,
     -73.9104159619131,
     40.85572707719664]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.16',
   'geometry': {'type': 'Point',
    'coordinates': [-73.91967159119565, 40.84789792606271]},
   'geometry_name': 'geom',
   'properties': {'name': 'Morris Heights',
    'stacked': 2,
    'annoline1': 'Morris',
    'annoline2': 'Heights',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.91967159119565,
     40.84789792606271,
     -73.91967159119565,
     40.84789792606271]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.17',
   'geometry': {'type': 'Point',
    'coordinates': [-73.89642655981623, 40.86099679638654]},
   'geometry_name': 'geom',
   'properties': {'name': 'Fordham',
    'stacked': 1,
    'annoline1': 'Fordham',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.89642655981623,
     40.86099679638654,
     -73.89642655981623,
     40.86099679638654]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.18',
   'geometry': {'type': 'Point',
    'coordinates': [-73.88735617532338, 40.84269615786053]},
   'geometry_name': 'geom',
   'properties': {'name': 'East Tremont',
    'stacked': 2,
    'annoline1': 'East',
    'annoline2': 'Tremont',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.88735617532338,
     40.84269615786053,
     -73.88735617532338,
     40.84269615786053]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.19',
   'geometry': {'type': 'Point',
    'coordinates': [-73.87774474910545, 40.83947505672653]},
   'geometry_name': 'geom',
   'properties': {'name': 'West Farms',
    'stacked': 2,
    'annoline1': 'West',
    'annoline2': 'Farms',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.87774474910545,
     40.83947505672653,
     -73.87774474910545,
     40.83947505672653]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.20',
   'geometry': {'type': 'Point',
    'coordinates': [-73.9261020935813, 40.836623010706056]},
   'geometry_name': 'geom',
   'properties': {'name': 'High  Bridge',
    'stacked': 1,
    'annoline1': 'Highbridge',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.9261020935813,
     40.836623010706056,
     -73.9261020935813,
     40.836623010706056]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.21',
   'geometry': {'type': 'Point',
    'coordinates': [-73.90942160757436, 40.819754370594936]},
   'geometry_name': 'geom',
   'properties': {'name': 'Melrose',
    'stacked': 1,
    'annoline1': 'Melrose',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.90942160757436,
     40.819754370594936,
     -73.90942160757436,
     40.819754370594936]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.22',
   'geometry': {'type': 'Point',
    'coordinates': [-73.91609987487575, 40.80623874935177]},
   'geometry_name': 'geom',
   'properties': {'name': 'Mott Haven',
    'stacked': 1,
    'annoline1': 'Mott Haven',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.91609987487575,
     40.80623874935177,
     -73.91609987487575,
     40.80623874935177]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.23',
   'geometry': {'type': 'Point',
    'coordinates': [-73.91322139386135, 40.801663627756206]},
   'geometry_name': 'geom',
   'properties': {'name': 'Port Morris',
    'stacked': 2,
    'annoline1': 'Port',
    'annoline2': 'Morris',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.91322139386135,
     40.801663627756206,
     -73.91322139386135,
     40.801663627756206]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.24',
   'geometry': {'type': 'Point',
    'coordinates': [-73.8957882009446, 40.81509904545822]},
   'geometry_name': 'geom',
   'properties': {'name': 'Longwood',
    'stacked': 1,
    'annoline1': 'Longwood',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.8957882009446,
     40.81509904545822,
     -73.8957882009446,
     40.81509904545822]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.25',
   'geometry': {'type': 'Point',
    'coordinates': [-73.88331505955291, 40.80972987938709]},
   'geometry_name': 'geom',
   'properties': {'name': 'Hunts Point',
    'stacked': 2,
    'annoline1': 'Hunts',
    'annoline2': 'Point',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.88331505955291,
     40.80972987938709,
     -73.88331505955291,
     40.80972987938709]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.26',
   'geometry': {'type': 'Point',
    'coordinates': [-73.90150648943059, 40.82359198585534]},
   'geometry_name': 'geom',
   'properties': {'name': 'Morrisania',
    'stacked': 1,
    'annoline1': 'Morrisania',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.90150648943059,
     40.82359198585534,
     -73.90150648943059,
     40.82359198585534]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.27',
   'geometry': {'type': 'Point',
    'coordinates': [-73.86574609554924, 40.821012197914015]},
   'geometry_name': 'geom',
   'properties': {'name': 'Soundview',
    'stacked': 1,
    'annoline1': 'Soundview',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.86574609554924,
     40.821012197914015,
     -73.86574609554924,
     40.821012197914015]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.28',
   'geometry': {'type': 'Point',
    'coordinates': [-73.85414416189266, 40.80655112003589]},
   'geometry_name': 'geom',
   'properties': {'name': 'Clason Point',
    'stacked': 2,
    'annoline1': 'Clason',
    'annoline2': 'Point',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.85414416189266,
     40.80655112003589,
     -73.85414416189266,
     40.80655112003589]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.29',
   'geometry': {'type': 'Point',
    'coordinates': [-73.81635002158441, 40.81510925804005]},
   'geometry_name': 'geom',
   'properties': {'name': 'Throgs Neck',
    'stacked': 1,
    'annoline1': 'Throgs Neck',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.81635002158441,
     40.81510925804005,
     -73.81635002158441,
     40.81510925804005]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.30',
   'geometry': {'type': 'Point',
    'coordinates': [-73.8240992675385, 40.844245936947374]},
   'geometry_name': 'geom',
   'properties': {'name': 'Country Club',
    'stacked': 2,
    'annoline1': 'Country',
    'annoline2': 'Club',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.8240992675385,
     40.844245936947374,
     -73.8240992675385,
     40.844245936947374]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.31',
   'geometry': {'type': 'Point',
    'coordinates': [-73.85600310535783, 40.837937822267286]},
   'geometry_name': 'geom',
   'properties': {'name': 'Parkchester',
    'stacked': 1,
    'annoline1': 'Parkchester',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.85600310535783,
     40.837937822267286,
     -73.85600310535783,
     40.837937822267286]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.32',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84219407604444, 40.8406194964327]},
   'geometry_name': 'geom',
   'properties': {'name': 'Westchester Square',
    'stacked': 2,
    'annoline1': 'Westchester',
    'annoline2': 'Square',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84219407604444,
     40.8406194964327,
     -73.84219407604444,
     40.8406194964327]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.33',
   'geometry': {'type': 'Point',
    'coordinates': [-73.8662991807561, 40.84360847124718]},
   'geometry_name': 'geom',
   'properties': {'name': 'Van Nest',
    'stacked': 2,
    'annoline1': 'Van',
    'annoline2': 'Nest',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.8662991807561,
     40.84360847124718,
     -73.8662991807561,
     40.84360847124718]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.34',
   'geometry': {'type': 'Point',
    'coordinates': [-73.85040178030421, 40.847549063536334]},
   'geometry_name': 'geom',
   'properties': {'name': 'Morris Park',
    'stacked': 1,
    'annoline1': 'Morris Park',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.85040178030421,
     40.847549063536334,
     -73.85040178030421,
     40.847549063536334]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.35',
   'geometry': {'type': 'Point',
    'coordinates': [-73.88845196134804, 40.85727710073895]},
   'geometry_name': 'geom',
   'properties': {'name': 'Belmont',
    'stacked': 1,
    'annoline1': 'Belmont',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.88845196134804,
     40.85727710073895,
     -73.88845196134804,
     40.85727710073895]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.36',
   'geometry': {'type': 'Point',
    'coordinates': [-73.91719048210393, 40.88139497727086]},
   'geometry_name': 'geom',
   'properties': {'name': 'Spuyten Duyvil',
    'stacked': 2,
    'annoline1': 'Spuyten',
    'annoline2': 'Duyvil',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.91719048210393,
     40.88139497727086,
     -73.91719048210393,
     40.88139497727086]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.37',
   'geometry': {'type': 'Point',
    'coordinates': [-73.90453054908927, 40.90854282950666]},
   'geometry_name': 'geom',
   'properties': {'name': 'North Riverdale',
    'stacked': 2,
    'annoline1': 'North',
    'annoline2': 'Riverdale',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.90453054908927,
     40.90854282950666,
     -73.90453054908927,
     40.90854282950666]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.38',
   'geometry': {'type': 'Point',
    'coordinates': [-73.8320737824047, 40.85064140940335]},
   'geometry_name': 'geom',
   'properties': {'name': 'Pelham Bay',
    'stacked': 2,
    'annoline1': 'Pelham',
    'annoline2': 'Bay',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.8320737824047,
     40.85064140940335,
     -73.8320737824047,
     40.85064140940335]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.39',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82620275994073, 40.82657951686922]},
   'geometry_name': 'geom',
   'properties': {'name': 'Schuylerville',
    'stacked': 1,
    'annoline1': 'Schuylerville',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82620275994073,
     40.82657951686922,
     -73.82620275994073,
     40.82657951686922]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.40',
   'geometry': {'type': 'Point',
    'coordinates': [-73.81388514428619, 40.821986118163494]},
   'geometry_name': 'geom',
   'properties': {'name': 'Edgewater Park',
    'stacked': 2,
    'annoline1': 'Edgewater',
    'annoline2': 'Park',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.81388514428619,
     40.821986118163494,
     -73.81388514428619,
     40.821986118163494]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.41',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84802729582735, 40.819014376988314]},
   'geometry_name': 'geom',
   'properties': {'name': 'Castle Hill',
    'stacked': 2,
    'annoline1': 'Castle',
    'annoline2': 'Hill',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84802729582735,
     40.819014376988314,
     -73.84802729582735,
     40.819014376988314]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.42',
   'geometry': {'type': 'Point',
    'coordinates': [-73.86332361652777, 40.87137078192371]},
   'geometry_name': 'geom',
   'properties': {'name': 'Olinville',
    'stacked': 1,
    'annoline1': 'Olinville',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.86332361652777,
     40.87137078192371,
     -73.86332361652777,
     40.87137078192371]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.43',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84161194831223, 40.86296562477998]},
   'geometry_name': 'geom',
   'properties': {'name': 'Pelham Gardens',
    'stacked': 2,
    'annoline1': 'Pelham',
    'annoline2': 'Gardens',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84161194831223,
     40.86296562477998,
     -73.84161194831223,
     40.86296562477998]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.44',
   'geometry': {'type': 'Point',
    'coordinates': [-73.91558941773444, 40.83428380733851]},
   'geometry_name': 'geom',
   'properties': {'name': 'Concourse',
    'stacked': 1,
    'annoline1': 'Concourse',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.91558941773444,
     40.83428380733851,
     -73.91558941773444,
     40.83428380733851]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.45',
   'geometry': {'type': 'Point',
    'coordinates': [-73.85053524451935, 40.82977429787161]},
   'geometry_name': 'geom',
   'properties': {'name': 'Unionport',
    'stacked': 1,
    'annoline1': 'Unionport',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.85053524451935,
     40.82977429787161,
     -73.85053524451935,
     40.82977429787161]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.46',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84808271877168, 40.88456130303732]},
   'geometry_name': 'geom',
   'properties': {'name': 'Edenwald',
    'stacked': 1,
    'annoline1': 'Edenwald',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84808271877168,
     40.88456130303732,
     -73.84808271877168,
     40.88456130303732]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.47',
   'geometry': {'type': 'Point',
    'coordinates': [-74.03062069353813, 40.625801065010656]},
   'geometry_name': 'geom',
   'properties': {'name': 'Bay Ridge',
    'stacked': 1,
    'annoline1': 'Bay Ridge',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-74.03062069353813,
     40.625801065010656,
     -74.03062069353813,
     40.625801065010656]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.48',
   'geometry': {'type': 'Point',
    'coordinates': [-73.99517998380729, 40.61100890202044]},
   'geometry_name': 'geom',
   'properties': {'name': 'Bensonhurst',
    'stacked': 1,
    'annoline1': 'Bensonhurst',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-73.99517998380729,
     40.61100890202044,
     -73.99517998380729,
     40.61100890202044]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.49',
   'geometry': {'type': 'Point',
    'coordinates': [-74.01031618527784, 40.64510294925429]},
   'geometry_name': 'geom',
   'properties': {'name': 'Sunset Park',
    'stacked': 2,
    'annoline1': 'Sunset',
    'annoline2': 'Park',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-74.01031618527784,
     40.64510294925429,
     -74.01031618527784,
     40.64510294925429]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.50',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95424093127393, 40.7302009848647]},
   'geometry_name': 'geom',
   'properties': {'name': 'Greenpoint',
    'stacked': 1,
    'annoline1': 'Greenpoint',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-73.95424093127393,
     40.7302009848647,
     -73.95424093127393,
     40.7302009848647]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.51',
   'geometry': {'type': 'Point',
    'coordinates': [-73.97347087708445, 40.59526001306593]},
   'geometry_name': 'geom',
   'properties': {'name': 'Gravesend',
    'stacked': 1,
    'annoline1': 'Gravesend',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-73.97347087708445,
     40.59526001306593,
     -73.97347087708445,
     40.59526001306593]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.52',
   'geometry': {'type': 'Point',
    'coordinates': [-73.96509448785336, 40.57682506566604]},
   'geometry_name': 'geom',
   'properties': {'name': 'Brighton Beach',
    'stacked': 2,
    'annoline1': 'Brighton',
    'annoline2': 'Beach',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-73.96509448785336,
     40.57682506566604,
     -73.96509448785336,
     40.57682506566604]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.53',
   'geometry': {'type': 'Point',
    'coordinates': [-73.94318640482979, 40.58689012678384]},
   'geometry_name': 'geom',
   'properties': {'name': 'Sheepshead Bay',
    'stacked': 2,
    'annoline1': 'Sheepshead',
    'annoline2': 'Bay',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-73.94318640482979,
     40.58689012678384,
     -73.94318640482979,
     40.58689012678384]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.54',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95743840559939, 40.61443251335098]},
   'geometry_name': 'geom',
   'properties': {'name': 'Manhattan Terrace',
    'stacked': 2,
    'annoline1': 'Manhattan',
    'annoline2': 'Terrace',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-73.95743840559939,
     40.61443251335098,
     -73.95743840559939,
     40.61443251335098]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.55',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95840106533903, 40.63632589026677]},
   'geometry_name': 'geom',
   'properties': {'name': 'Flatbush',
    'stacked': 1,
    'annoline1': 'Flatbush',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-73.95840106533903,
     40.63632589026677,
     -73.95840106533903,
     40.63632589026677]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.56',
   'geometry': {'type': 'Point',
    'coordinates': [-73.94329119073582, 40.67082917695294]},
   'geometry_name': 'geom',
   'properties': {'name': 'Crown Heights',
    'stacked': 2,
    'annoline1': 'Crown',
    'annoline2': 'Heights',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Brooklyn',
    'bbox': [-73.94329119073582,
     40.67082917695294,
     -73.94329119073582,
     40.67082917695294]}}
             ]}

In [74]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
Latitude=[]
Longitude=[]
for i in range(len(toronto_data)):
    
    Latitude.append(neighborhoods['Latitude'][i])  
    Longitude.append(neighborhoods['Longitude'][i])  
    
toronto_data['Latitude']=Latitude 
toronto_data['Longitude']=Longitude


In [75]:
from sklearn.cluster import KMeans

kclusters = 3

toronto_data1 = toronto_data.drop('Neighbourhoodall', 1) 
toronto_data1 = toronto_data1.drop('Postcode', 1) 
toronto_data1 = toronto_data1.drop('Borough', 1) 


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_data1)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [76]:
# add clustering labels
toronto_data_label=toronto_data
toronto_data_label.insert(0, 'Cluster Labels', kmeans.labels_)



toronto_data_label.head() 

,Cluster Labels,Postcode,Borough,Neighbourhoodall,Latitude,Longitude
0,1,M4E,East Toronto,The Beaches,40.894705,-73.847201
1,1,M4K,East Toronto,"The Danforth West ,Riverdale",40.874294,-73.829939
2,1,M4L,East Toronto,"The Beaches West ,India Bazaar",40.887556,-73.827806
3,0,M4M,East Toronto,Studio District,40.895437,-73.905643
4,0,M4N,Central Toronto,Lawrence Park,40.890834,-73.912585


In [84]:
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data_label['Latitude'], toronto_data_label['Longitude'], toronto_data_label['Neighbourhoodall'], toronto_data_label['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters